<a href="https://colab.research.google.com/github/marlon-br/glow-tts-colab/blob/master/MozillaWavsToText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install deepspeech==0.7.3
!apt -qq install -y sox

     |████████████████████████████████| 9.7MB 2.7MB/s 
  Found existing installation: deepspeech 0.6.0
    Uninstalling deepspeech-0.6.0:
      Successfully uninstalled deepspeech-0.6.0


sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.


In [5]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.7.3/deepspeech-0.7.3-models.pbmm
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.7.3/deepspeech-0.7.3-models.scorer

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   639  100   639    0     0    936      0 --:--:-- --:--:-- --:--:--   935
100  180M  100  180M    0     0  13.0M      0  0:00:13  0:00:13 --:--:-- 16.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   641  100   641    0     0    956      0 --:--:-- --:--:-- --:--:--   956
100  909M  100  909M    0     0  15.6M      0  0:00:57  0:00:57 --:--:-- 16.1M


In [6]:
!deepspeech --model deepspeech-0.7.3-models.pbmm --scorer deepspeech-0.7.3-models.scorer --audio "/content/drive/My Drive/wavs-vctk/speakers/p229/p229_001.wav"

Loading model from file deepspeech-0.7.3-models.pbmm
TensorFlow: v1.15.0-24-gceb46aa
DeepSpeech: v0.7.3-0-g8858494
2020-06-18 09:19:57.388496: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Loaded model in 0.00872s.
Loading scorer from files deepspeech-0.7.3-models.scorer
Loaded scorer in 0.000214s.
Running inference.
please call della
Inference took 1.446s for 2.136s audio file.


In [8]:
import deepspeech

model_file_path = 'deepspeech-0.7.3-models.pbmm'
#beam_width = 500
model = deepspeech.Model(model_file_path)

# Add language model for better accuracy
#lm_file_path = 'deepspeech-0.6.0-models/lm.binary'
#trie_file_path = 'deepspeech-0.6.0-models/trie'
#lm_alpha = 0.75
#lm_beta = 1.85
#model.enableDecoderWithLM(lm_file_path, trie_file_path, lm_alpha, lm_beta)

In [9]:
from typing import Tuple
import wave
import sox
import numpy as np

temp_filename = '/content/output.wav'
tfm = sox.Transformer()
tfm.set_output_format(rate=16000)

def read_wav_file(filename) -> Tuple[bytes, int]:

    sample_rate = sox.file_info.sample_rate(filename)

    if (sample_rate != 16000):
        print("resampling")

        
        status = tfm.build(input_filepath=filename,
                            output_filepath=temp_filename)
        
        filename = temp_filename

    with wave.open(filename, 'rb') as w:
        rate = w.getframerate()
        frames = w.getnframes()
        buffer = w.readframes(frames)

    return buffer, rate

In [ ]:
import numpy as np
import os


!rm -r /content/results
!mkdir /content/results

files_dir = "p229"
full_files_path = os.path.join("/content/drive/My Drive/wavs-vctk/speakers", files_dir)

new_filename = os.path.join("/content/results", "speaker_p229.csv")

file = open(new_filename,"w")


for i, filename in enumerate(os.listdir(full_files_path)[:3]):
    print(filename)
    full_path = os.path.join(full_files_path, filename)


    buffer, rate = read_wav_file(full_path)
    data16 = np.frombuffer(buffer, dtype=np.int16)

    out = model.sttWithMetadata(data16)
    print(out)
    #output_string = model.stt(data16)

    #new_item = str(i) + "," + os.path.join(files_dir, filename) + "," + output_string + "~\n"
    #print(new_item)      
    #file.write(new_item)

file.close() 

In [ ]:
import numpy as np

def deepspeech_batch_stt(filename: str, lang: str, encoding: str) -> str:
    buffer, rate = read_wav_file(filename)
    data16 = np.frombuffer(buffer, dtype=np.int16)
    return model.stt(data16)

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('deepspeech-batch-stt: "{}"'.format(
        deepspeech_batch_stt(t['filename'], t['lang'], t['encoding'])
    ))